In [1]:
import numpy as np
import math
import pandas as pd
import itertools
import random

I2_N10_T30_C200_0 -> I2_N10_T30_C250_0 \
I2_N10_T30_C400_0 -> I2_N10_T30_C350_0 \
I2_N10_T100_C400_0 -> I2_N10_T100_C350_0 \
I2_N10_T100_C200_0 -> I2_N10_T100_C250_0 \

In [ ]:
dfInstances = pd.DataFrame(columns=['name', 'solved', 'policy', 'objVal','runTime','gap',
                                    'Z1','Z2','Z3','Z4','Z5', 'EVV','EZ1','EZ2','EZ3','EZ4','EZ5', 'VSS' ])
for n in [5,7,9]:
    for m in [2,3]:
        for Tmax in [15,30,45]:
            for C in [100,150,200]:
                for Depot in ['Central', 'LowerLeft']:
                    for seed in [0,1,2,3,4]:
                        #GENERATE X, y
                        random.seed(seed)
                        name =	'I3_N{}_M{}_T{}_C{}_Dep{}_s{}'.format(n,m,Tmax,C,Depot,seed)
                        dfInstances.loc[len(dfInstances)] = [name, False, 'STOCHASTIC', -1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]
                        LT =	3
                        LS =	5
                        #print(f"base capacity = {C}")
                        print (f'generating instance named {name}')
                        p =	10

                        Qmax =	10000
                        M =	10000
                        service_time =	0
                        D =	50   #TODO D = 20
                        delta = 10 #

                        minx = 0
                        miny = 0
                        maxx = 10
                        maxy = 10


                        df_param = pd.DataFrame()
                        df_param['param'] = ['name', 'n', 'LT', 'm', 'LS', 'C', 'p', 'Tmax', 'Qmax', 'M', 'service_time', 'D', 'Depot', 'seed']
                        df_param['value'] = [name, n, LT, m, LS, C, p, Tmax, Qmax, M, 0, D, Depot, seed]

                        df_time_windows = pd.DataFrame()
                        df_time_windows['t'] = [1,2,3]
                        if Tmax ==	15:
                            df_time_windows['e'] =  [0, 5, 10]
                            df_time_windows['l'] = [5, 10, 15]
                        if Tmax ==	30:
                            df_time_windows['e'] =  [0, 10, 20]
                            df_time_windows['l'] = [10, 20, 30]
                        elif Tmax == 45:
                            df_time_windows['e'] =  [0, 15, 30]
                            df_time_windows['l'] = [15, 30, 45]

                        df_edges = pd.DataFrame()
                        #generate example data
                        df_edges['i'] = [0,    0,    1, 1, 2, 2]
                        df_edges['j'] = [1,    2,    0, 2, 1, 0]
                        df_edges['c'] = [10.2, 11.2, 1, 2, 3, 4]
                        df_edges['t'] = [10.2, 11.2, 1, 2, 3, 4]

                        # GENERATE NODES AND LOCATIONS
                        rndy = [round(random.random()*(maxy - miny) + miny,2) for i in range(n)]
                        rndx = [round(random.random()*(maxx - minx) + minx,2) for i in range(n)]

                        if Depot == 'LowerLeft':
                            rndy = [0] + rndy
                            rndx = [0] + rndx
                        if Depot == 'Central':
                            rndy = [5] + rndy
                            rndx = [5] + rndx
                        df_nodes = pd.DataFrame()
                        df_nodes['i'] = [i for i in range(n + 1)]
                        df_nodes['x'] = rndx
                        df_nodes['y'] = rndy
                        df_nodes['service_time'] = [service_time for i in range(n+1)]

                        # GENERATING PATTERNS FOR CUSTOMER DEMAND AND DELTA
                        patternCustomersDict = {
                            1: {'demand' : 'low',
                                'demand_value' : 0.5,
                                'delta'  : ['high', 'low', 'low'],
                                'delta_value'  : [1.5, 0.5, 0.5]},
                            2: {'demand' : 'average',
                                'demand_value' : 1,
                                'delta'  : ['average', 'high', 'low'],
                                'delta_value'  : [1, 1.5, 0.5]},
                            3: {'demand' : 'high',
                                'demand_value' : 1.5,
                                'delta'  : ['low', 'low', 'average'],
                                'delta_value'  : [0.5, 0.5, 1]}
                        }

                        maxDemand = D*1.5  
                        minDemand = D*0.5  
                        maxdelta = delta*1.5
                        mindelta = delta*0.5

                        #GENERATING DEMAND AND DELTA

                        #rnd_minDelta = [round(random.random()*mindelta,2)  for i in range(LS*LT*n)]
                        #rnd_maxDelta = [round(random.random()*maxdelta, 2) for i in range(LS*LT*n)]
                        #rndDelta = [round(random.random()*(rnd_maxDelta[i] - rnd_minDelta[i]) + rnd_minDelta[i],2) for i in range(LS*LT*n)]
                        rndDelta = [round(random.random()*(maxdelta - mindelta) + mindelta,2) for i in range(LS*LT*n)]
                        rndDemand = [round(random.random()*(maxDemand - minDemand) + minDemand,2) for i in range(n)]
                        i_d = 0
                        for i in range(n):
                            PatternCustomers =	random.randint(1, len(patternCustomersDict))
                            pc = patternCustomersDict[PatternCustomers]
                            rndDemand[i] = rndDemand[i] * pc['demand_value'] #TODO correggere domanda 
                            for t in range (LT):
                                delta_current = pc['delta_value'][t]
                                for s in range (LS):
                                    rndDelta[i_d] = rndDelta[i_d]* delta_current #TODO rivedere delta
                                    i_d += 1

                        indices = [(i, t, s) for (i,t,s) in itertools.product(range(1, n+1), range(1, LT+1), range(1, LS+1)) ]
                        df_delta = pd.DataFrame()
                        df_demand = pd.DataFrame()
                        df_delta['j'] = [i[0] for i in indices]
                        df_delta['t'] = [i[1] for i  in indices]
                        df_delta['s'] = [i[2] for i in indices]
                        df_delta['delta'] = rndDelta
                        df_demand['j'] = [j for j in range(1,n+1)]
                        df_demand['d'] = rndDemand

                        # writing output

                        filename = 'instances/' + name +  '.xlsx'
                        with pd.ExcelWriter(filename) as writer:
                            df_param.to_excel(writer, sheet_name='params', index=None)
                            df_nodes.to_excel(writer, sheet_name='nodes', index=None)
                            #df_delta.to_excel(writer, sheet_name='delta', index=None)
                            df_edges.to_excel(writer, sheet_name='edges', index=None)
                            df_time_windows.to_excel(writer, sheet_name='time_windows', index=None)
                            df_demand.to_excel(writer, sheet_name='demand', index=None)
                            df_delta.to_excel(writer, sheet_name='delta', index=None)
                            
                            # COMPUTNG THE AVERAGE SCENARIO
                            dfmean = df_delta.groupby(['j', 't']).agg({'delta' : 'mean'})
                            dfmean = dfmean.reset_index()
                            #dfmean['s'] = [1 for i in len(dfmean)]
                            dfmean.insert(loc=2, column='s', value=[1 for i in range(n*LT)])
                            # writing average scenario
                            name_mean = name + "_mean"
                            df_param.loc[df_param['param'] == 'name', 'value'] = name_mean
                            df_param.loc[df_param['param'] == 'LS', 'value'] = 1
                            filename = 'instances/' + name_mean +  '.xlsx'
                            with pd.ExcelWriter(filename) as writer:
                                df_param.to_excel(writer, sheet_name='params', index=None)
                                df_nodes.to_excel(writer, sheet_name='nodes', index=None)
                                #df_delta.to_excel(writer, sheet_name='delta', index=None)
                                df_edges.to_excel(writer, sheet_name='edges', index=None)
                                df_time_windows.to_excel(writer, sheet_name='time_windows', index=None)
                                df_demand.to_excel(writer, sheet_name='demand', index=None)
                                dfmean.to_excel(writer, sheet_name='delta', index=None)
                                #df_delta.to_excel(writer, sheet_name='delta', index=None)


                            #dfmean.to_excel('scenario_mean_S4.xlsx', sheet_name='delta', index=None)
dfInstances.to_excel('Instances_list.xlsx', index=None)


generating instance named I3_N5_M2_T15_C100_DepCentral_s0
generating instance named I3_N5_M2_T15_C100_DepCentral_s1
generating instance named I3_N5_M2_T15_C100_DepCentral_s2
generating instance named I3_N5_M2_T15_C100_DepCentral_s3
generating instance named I3_N5_M2_T15_C100_DepCentral_s4
generating instance named I3_N5_M2_T15_C100_DepLowerLeft_s0
generating instance named I3_N5_M2_T15_C100_DepLowerLeft_s1
generating instance named I3_N5_M2_T15_C100_DepLowerLeft_s2
generating instance named I3_N5_M2_T15_C100_DepLowerLeft_s3
generating instance named I3_N5_M2_T15_C100_DepLowerLeft_s4
generating instance named I3_N5_M2_T15_C150_DepCentral_s0
generating instance named I3_N5_M2_T15_C150_DepCentral_s1
generating instance named I3_N5_M2_T15_C150_DepCentral_s2
generating instance named I3_N5_M2_T15_C150_DepCentral_s3
generating instance named I3_N5_M2_T15_C150_DepCentral_s4
generating instance named I3_N5_M2_T15_C150_DepLowerLeft_s0
generating instance named I3_N5_M2_T15_C150_DepLowerLeft_s1


In [3]:

dfmean

,j,t,s,delta
0,1,1,1,10.324
1,1,2,1,16.800
2,1,3,1,4.682
3,2,1,1,5.694
4,2,2,1,5.751
5,2,3,1,8.506
6,3,1,1,4.458
7,3,2,1,4.652
8,3,3,1,9.666
9,4,1,1,13.065
